In [10]:
open Microsoft.Quantum.Extensions.Diagnostics;
operation Qrng() : Result {
    using (qubit = Qubit()) {
        Message("Here is what the simulator uses to record a qubit in the 0 state:");
        DumpRegister((), [qubit]);

        Message(" ");
        Message("Here is what the simulator uses to record a superposition state:");
        H(qubit);
        DumpRegister((), [qubit]);
        
        return MResetZ(qubit);
    }
}

Qrng

In [11]:
%simulate Qrng

Here is what the simulator uses to record a qubit in the 0 state:
# wave function for qubits with ids (least to most significant): 0
0:	1	0
1:	0	0
 
Here is what the simulator uses to record a superposition state:
# wave function for qubits with ids (least to most significant): 0
0:	0.707106781186548	0
1:	0.707106781186548	0


Zero

In [13]:
operation ZeroOracle(control : Qubit, target : Qubit) : Unit {
}

operation OneOracle(control : Qubit, target : Qubit) : Unit {
    X(target);
}

operation IdOracle(control : Qubit, target : Qubit) : Unit {
    CNOT(control, target);
}

operation NotOracle(control : Qubit, target : Qubit) : Unit {
    X(control);
    CNOT(control, target);
    X(control);
}

IdOracle, NotOracle, OneOracle, ZeroOracle

In [ ]:
operation IsOracleBalanced(
    oracle : ((Qubit, Qubit) => Unit)) : Bool {
mutable result = Zero;
using ((control, target) = (Qubit(), Qubit())) { //①
H(control); //②
X(target);
H(target);
oracle(control, target); //③
H(target); //④
X(target);
set result = MResetX(control); //⑤
}
return result == One;
}